# Imports

In [1]:
%load_ext autoreload
%autoreload 1
%load_ext lab_black

import os

os.chdir("/home/roblesi/git/document_information_extraction/")
os.getcwd()

'/home/roblesi/git/document_information_extraction'

In [2]:
import requests
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.retriever.sparse import ElasticsearchRetriever  # import BM25 retriever

/home/roblesi/anaconda3/envs/doc_info/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [3]:
from src.retriever.retriever_input_articles import (
    make_input_article_generator,
    row_to_article,
)

# Start document store
We will use request to comunicate with ElasticSearch

In [4]:
# Check status
res = requests.get("http://localhost:9200/_cluster/health")
res.json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 3,
 'active_shards': 3,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 60.0}

In [5]:
document_store = ElasticsearchDocumentStore(
    host="localhost", username="", password="", index="squad_docs"
)

/home/roblesi/anaconda3/envs/doc_info/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [6]:
article_generator = make_input_article_generator(n_sample_texts=None)

In [7]:
docs = []
for row in article_generator:
    page_id, article = row_to_article(row)
    docs += [{"content": article, "meta": {"pageid": page_id}}]

In [8]:
docs[1000]

{'content': " \nPay to surf (PTS) is a popular business model in the late 1990s, but it experiences a significant decline following the dot-com crash. Users of the internet install programs that tracked their browsing activity and displayed targeted advertisements.  Advertising revenue would be used to financially compensate users for their time spent browsing the web.  Today, there are still a few PTS companies operating.\n\n\n\nA PTS company would provide a smaller program to be installed on a member's computer.  Advertisers' banner ads were then displayed while the member was browsing the web. Since the company's Viewbar software tracked websites that the user visited, the PTS company was able to deliver targeted ads for their advertisers. Advertisers paid the company a small amount (typically US$0.50) for every hour of a member's surfing.\n\nMembers were usually limited on the amount of time per month for which they would be paid to surf (typically 20 hours). However, PTS companies

In [9]:
document_store.write_documents(docs)

POST http://localhost:9200/_bulk?refresh=wait_for [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/home/roblesi/anaconda3/envs/doc_info/lib/python3.8/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/roblesi/anaconda3/envs/doc_info/lib/python3.8/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/roblesi/anaconda3/envs/doc_info/lib/python3.8/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/roblesi/anaconda3/envs/doc_info/lib/python3.8/site-packages/elasticsearch/connection/http_urllib3.py", line 245, in perform_request
    response = self.pool.urlopen(
  File "/home/roblesi/anaconda3/envs/doc_info/lib/python3.8/site-packages/urllib3/co

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7ff622cee790>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7ff622cee790>: Failed to establish a new connection: [Errno 111] Connection refused)

In [ ]:
retriever = ElasticsearchRetriever(document_store)

In [ ]:
query = """Osteopathic clinic"""

In [ ]:
retrieved = retriever.retrieve(query)
retrieved

# Delete indeces

In [ ]:
res = requests.post(
    "http://localhost:9200/squad_docs/_delete_by_query",
    json={"query": {"match_all": {}}},
)
res

In [ ]:
res = requests.post(
    "http://localhost:9200/squad_docs/_count",
    json={"query": {"match_all": {}}},
)
res.json()